In [7]:
#type1 依斷字後符合apart cut及cpart 的數目選高的當答案,如果,完全不符合,就用猜的
#type2 依斷字後符合apart cut及cpart 的數目選低的當答案
import sqlite3
import pandas as pd
import random
import os
import sys
import csv
import spiltc
import jieba
#jieba.set_dictionary('jieba_dict/dict.txt.big')
jieba.set_dictionary('jieba_dict/dict_tw.txt')

def genbtype(conn):
    try:        
        sql="update Bpart set Qtype=1"  
        conn.execute(sql )
    except Exception as e:
        print("Failed update Qtype 1 : "+ str(e)) 
    conn.commit()
    sql="update Bpart set Qtype=2 WHERE sentence like '%非%' and sentence not like '%非常%' or sentence like '%不包%' or sentence like '%不是%' or  sentence like '%不屬%' "  
    try:        
        conn.execute(sql )
    except Exception as e:
        print("Failed update qtype 2: "+ str(e)) 
    conn.commit()
 
    print("Qtype1 positive, qtype2 negative over")
#產生答案
def gencsv(conn):
    try:
        sql="SELECT  ID,  Answer FROM Cpart order by ID;"
        pdgrand=pd.read_sql(sql,conn)
        print('total answer no=',len(pdgrand))
        pdgrand.to_csv(r'./answer.csv',columns=['ID','Answer'],index=False,sep=',')
        conn.commit()
    except Exception as e:
        print('gencsv error'+str(e))
#產生答案
def genscore(conn):
    try:
        sql="SELECT ID, COALESCE( score1,0) as score1, COALESCE(score2,0) as score2,\
        COALESCE(score3,0) as score3 , COALESCE(score4,0) as score4 FROM Cpart ; "
        pdgrand=pd.read_sql(sql,conn)
        print('score no=',len(pdgrand))
        pdgrand.to_csv(r'./score.csv',columns=['ID','score1','score2','score3','score4'],index=False,sep=',')
        conn.commit()
    except Exception as e:
        print('genscore err'+str(e))
def addpatch(conn):
##################################
#check 合併後少的檔案
##################################
  
  #蒐集目錄下wav檔名單

    try:
        wavlist = []
        directory = 'A/' 
        objects = os.listdir(directory)
        for filelist in objects:
            if filelist.find('.wav') != -1:
                filelist='C'+filelist[1:]
                wavlist.append(filelist.upper())
        print('afgterfilelist')
        wavlist_set=set(wavlist)

  #取得DB中的已辨識名單

        cursor = conn.cursor()
        sql="SELECT Filename from  Cpart  ;" 
        cursor.execute(sql)
        dbvalues = list(cursor.fetchall())
        ldbvalues=[]
        sdbvalues=set()
        for i in dbvalues:
            ldbvalues.append(i[0])
            sdbvalues=set(ldbvalues)

        #比較目錄名單及DB名單的差異
        diff=wavlist_set-sdbvalues
        #print(diff)
        #df=pd.Series(list[diff]) 
        #df.to_csv('lost.csv', sep='\t', encoding='utf-8')
        #開啟新DB，將缺少的名單存入
        print('add patch from A folder'+str(len(diff)))
        for filename in diff:
            try:
                ID=int(filename[4:8])
                sql="insert into Cpart (filename,ID) VALUES('%s',%d)"  %(filename,ID)
                #print(sql)
                conn.execute(sql)
            except Exception as e:
                print("Failed insert db: "+ str(e)) 
                print(sql)
        
        conn.commit() 
    except Exception as e:
        print('addpatch err'+str(e))
   
#answettype =0 表示答案預設0,否則用random
def initanswer(conn,answertype):
    try:
        sql="select * from Cpart"
        pdgrand=pd.read_sql(sql,conn)
        answer=0
        for i in range(len(pdgrand)):
            filename=pdgrand.filename.values[i]
            if answertype!=0:
                answer=random.randint(1,4)   
            sql="update Cpart set Answer=%d where filename='%s'; " %(answer,filename)
            #print(sql)
            conn.execute(sql )
        conn.commit()
    except Exception as e:
        print('init answer err'+str(e))

def gentype2score(conn):
    try:
        sql="SELECT C.ID,c.qno as qno,count(*) as score \
            FROM Ccut c, acut a,bpart b where a.id=c.id and a.cut=c.cut and a.id=b.id \
            and b.Qtype= 2 group by c.ID,qno order by c.ID,count(*) DESC"     
        pdgrand=pd.read_sql(sql,conn)
        
        for i in range(len(pdgrand)):
            ID=pdgrand.ID.values[i]
            qno=pdgrand.qno.values[i]
            score=-pdgrand.score.values[i] 
            sql="update Cpart set score%d=%d where ID =%d " %(qno,score,ID)
            conn.execute(sql)
        conn.commit()
    except Exception as e:
        print('gentype2score err '+str(e))
      #print(sql)
    
        
def gentype2ans(conn):
  try:  
    gentype2score(conn)
    sql='SELECT c.ID, COALESCE(score1,0)score1, COALESCE(score2,0)score2,\
      COALESCE(score3,0)score3, COALESCE(score4,0)score4 FROM Cpart c,\
      Bpart B where c.id=b.id and qtype=2;'
    pdgrand=pd.read_sql(sql,conn)

    for i in range(len(pdgrand)):
      ID=pdgrand.ID.values[i]
      score1=pdgrand.score1.values[i] 
      score2=pdgrand.score2.values[i] 
      score3=pdgrand.score3.values[i] 
      score4=pdgrand.score4.values[i] 
      maxanswer=score1
      answer=1
      if score2>maxanswer:
        maxanswer=score2
        answer=2
      elif score3>maxanswer:
        maxanswer=score3
        answer=3
      elif score4>maxanswer:
        answer=4
      sql="update Cpart set Answer=%d where ID =%d " %(answer,ID)
     # print(sql)
      conn.execute(sql )
    conn.commit()
  except Exception as e:
    print("Failed gentype2ans error "+ str(e))

#正向的問題
def gentype1ans(conn):
  try:  
    sql="SELECT C.ID,c.qno as qno,count(*) as score \
      FROM Ccut c, acut a,bpart b where a.id=c.id and a.cut=c.cut and a.id=b.id \
      and b.Qtype=1 group by c.ID,qno order by c.ID,count(*) DESC "
#test將b 加進去     
   # sql="SELECT C.ID,c.qno as qno,count(*) as score \
    # FROM Ccut c, (select * from acut union select * from bcut) a,bpart b where a.id=c.id and a.cut=c.cut and a.id=b.id \
     # and b.Qtype=1 group by c.ID,qno order by c.ID,count(*) DESC "
    #print(sql)
    pdgrand=pd.read_sql(sql,conn)
    oldID=0
    for i in range(len(pdgrand)):
      ID=pdgrand.ID.values[i]
      qno=pdgrand.qno.values[i]
      score=pdgrand.score.values[i] 
      sql="update Cpart set score%d=%d where ID =%d " %(qno,score,ID)
      #print(sql)
      conn.execute(sql )
      #sql="update list_A set Answer=%d where substr(filename,5,4) ='%s' " %(qno,filename[5:4])
      if ID!=oldID:
        sql="update Cpart set Answer=%d where ID =%d " %(qno,ID)
      #print(sql)
        conn.execute(sql )
        oldID=ID
    conn.commit()
  except Exception as e:
    print("Failed gentype2ans error "+ str(e))
    
def cutab(conn,part,cutmethod):
    try:
        sql="delete FROM %scut;" %(part)
        #print(sql)
        conn.execute(sql )
        conn.commit()
    except Exception as e:
        print('delete error'+ str(e))
    try:
        sql="SELECT ID, sentence FROM %spart;" %(part)
        pdgrand=pd.read_sql(sql,conn)
        for i in range(len(pdgrand)):
            ID=pdgrand.ID.values[i]
            sentence=pdgrand.sentence.values[i]
            if sentence is None:
                print('none'+str(ID))
                continue
            #print(station)
            words=sentence
            if cutmethod ==1:
                words = jieba.cut(sentence, cut_all=False)
            for j in words:
                try:
                    sql="insert into %scut (ID, cut) VALUES( %d,'%s')"  %(part,ID, j)
                    # print(sql)
                    if j!=' ':
                        conn.execute(sql )
                except Exception as e:
                    print("Failed insert "+ str(e))
        conn.commit()
    except Exception as e:
                    print("Failed cut "+ str(e))
def cutcq(conn,ID,words,qno,cutmethod):
    try:
        if cutmethod ==1:
            words = jieba.cut(words, cut_all=False)
        for j in words:
            sql="insert into Ccut (ID, cut,qno) VALUES(%d, '%s',%d )"  %(ID, j,qno)
            if j!=' ':
                conn.execute(sql )
    except Exception as e:
        print('cutcq ID='+str(ID)+' no='+str(qno)+str(e))      
def cutc(conn,cutmethod):
    try:
        sql="delete FROM Ccut;" 
        print(sql)
        conn.execute(sql )
        conn.commit()
    except Exception as e:
        print('cutc error'+str(e))
    try:
        sql="SELECT ID, coalesce(sentence,''),Q1,Q2,Q3,Q4 FROM Cpart;"
        pdgrand=pd.read_sql(sql,conn)
        for i in range(len(pdgrand)):            
            ID=pdgrand.ID.values[i]
            sentence=pdgrand.Q1.values[i]
            #print('test'+sentence)
            if sentence is None:
                print('none q1 '+str(ID))
            else:
               cutcq(conn,ID,sentence,1,cutmethod)                                                              
            sentence=pdgrand.Q2.values[i]
            if sentence is None:
                print('none q2 '+str(ID))
            else:
               cutcq(conn,ID,sentence,2,cutmethod)
            sentence=pdgrand.Q3.values[i]
            if sentence is None:
                print('none q3 '+str(ID))
            else:
               cutcq(conn,ID,sentence,3,cutmethod)                                                                 
            sentence=pdgrand.Q4.values[i]
            if sentence is None:
                print('none q4 '+str(ID))
            else:
               cutcq(conn,ID,sentence,4,cutmethod)                                                                   
        conn.commit()
    except Exception as e:
        print("Failed insert Ccut: "+ str(e)) 
        print(sql)

try:    
    conn = sqlite3.connect('grand.db')
    cutab(conn,'A',1)
    print('cut A finish')
    cutab(conn,'B',1)
    print('cut B finish')
    spiltc.spiltc(conn)
    print('spiltc finish')   
    cutc(conn,1)
    print('cutc fiish')  
 
    #把 cpart補到1500
    addpatch(conn)
    print('補完了')
    #1是random,0是起始為0
    initanswer(conn,1)
    print('init finish')
    genbtype(conn)
    #for bpart type1正向問題
    gentype1ans(conn)
    print('answer1 complete')
    #for bpart type2反向問題
    gentype2ans(conn)
    print('answer2 complete')
    gencsv(conn)
    print('generate CSV file')
    genscore(conn)
    print('generate score file')
    conn.commit()
    conn.close() 
except Exception as e:
    print("Failed main error "+ str(e))


Building prefix dict from D:\aischool\2019S\semi-finals-2019\data\wav\jieba_dict\dict_tw.txt ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.u9d815a92b9225987d9fe8bdf37e7eec8.cache
Loading model cost 1.2660722732543945 seconds.
Prefix dict has been built succesfully.


cut A finish
cut B finish
nfound= 692 total= 1000
spiltc finish
delete FROM Ccut;
cutc fiish
afgterfilelist
add patch from A folder0
補完了
init finish
Qtype1 positive, qtype2 negative over
answer1 complete
answer2 complete
total answer no= 1000
generate CSV file
score no= 1000
generate score file


In [ ]:
import sqlite3
def genbtype(conn):
  sql="update Bpart set Qtype=1"  
  try:        
        conn.execute(sql )
  except Exception as e:
        print("Failed update Qtype 1 : "+ str(e)) 
  conn.commit()
  sql="update Bpart set Qtype=2 WHERE sentence like '%非%' and sentence not like '%非常%' or sentence like '%不包%' or sentence like '%不是%' or  sentence like '%不屬%' "  
  try:
        
        conn.execute(sql )
  except Exception as e:
        print("Failed update qtype 2: "+ str(e)) 
  conn.commit()
  conn.close()
  print("Qtype1 positive, qtype2 negative over")

In [ ]:
conn.close()

In [ ]:

#輸出答案
import jieba  
import sqlite3
import numpy as np
import pandas as pd
import random
conn = sqlite3.connect('grand.db')
#sql="SELECT cast(substr(filename,5,4) as int)  as ID,  Answer FROM Cpart;"
sql="SELECT cast(substr(filename,5,4) as int)  as ID,1 as Answer,filename FROM list_A order by filename"
#print(sql)
pdgrand=pd.read_sql(sql,conn)
for i in range(len(pdgrand)):
   answer=random.randint(1,4)  
   filename=pdgrand.filename.values[i]
   pdgrand.Answer.values[i]=answer
   sql="update list_A set Answer=%d where filename='%s'; " %(answer,filename)
      #print(sql)
   conn.execute(sql )
conn.commit()
conn.close()
print(len(pdgrand))

#pdgrand.to_csv(r'./answ0721sub1.csv',columns=['ID','Answer'],index=False,sep=',')



In [ ]:

#輸出答案
import jieba  
import sqlite3
import numpy as np
import pandas as pd
import random
conn = sqlite3.connect('grand.db')
def gencsv(conn):
  sql="SELECT  ID,  Answer FROM Cpart order by ID;"
  pdgrand=pd.read_sql(sql,conn)
  print(len(pdgrand))
  pdgrand.to_csv(r'./answ0721sub6.csv',columns=['ID','Answer'],index=False,sep=',')
  conn.commit()
gencsv(conn)
conn.close() 

In [ ]:
a=0.987654321


In [ ]:

#起始答案
import jieba  
import sqlite3
import numpy as np
import pandas as pd
import random
conn = sqlite3.connect('grand.db')

sql="select * from Cpart"
pdgrand=pd.read_sql(sql,conn)

for i in range(len(pdgrand)):
   filename=pdgrand.filename.values[i]
   answer=random.randint(1,4)   
   print(answer)
   sql="update Cpart set Answer=%d where filename='%s'; " %(answer,filename)
      #print(sql)
   conn.execute(sql )
conn.commit()
conn.close()

In [ ]:

#起始答案
import jieba  
import sqlite3
import numpy as np
import pandas as pd
import random
conn = sqlite3.connect('grand.db')

sql="select * from Cpart"
pdgrand=pd.read_sql(sql,conn)

for i in range(len(pdgrand)):
   filename=pdgrand.filename.values[i]
   ID=int(filename[4:8])
   #answer=random.randint(1,4)   
   #print(answer)
   sql="update Cpart set ID='%s' where filename='%s'; " %(ID,filename)
      #print(sql)
   conn.execute(sql )
conn.commit()
conn.close()
print('finish')

In [ ]:
a='C0000554.wav'
print(a[4:8])
print(int(a[4:8]))

In [ ]:
#依斷字後符合apart cut及cpart 的數目選高的當答案,如果,完全不符合,就沒答案
import sqlite3
import pandas as pd
import random
import os
import sys
import csv

conn = sqlite3.connect('grand.db')
def genbtype(conn):
  sql="update Bpart set Qtype=1"  
  try:        
        conn.execute(sql )
  except Exception as e:
        print("Failed update Qtype 1 : "+ str(e)) 
  conn.commit()
  sql="update Bpart set Qtype=2 WHERE sentence like '%非%' and sentence not like '%非常%' or sentence like '%不包%' or sentence like '%不是%' or  sentence like '%不屬%' "  
  try:
        
        conn.execute(sql )
  except Exception as e:
        print("Failed update qtype 2: "+ str(e)) 
  conn.commit()
 
  print("Qtype1 positive, qtype2 negative over")
#產生答案
def gencsv(conn):
  sql="SELECT  ID,  Answer FROM Cpart order by ID;"
  pdgrand=pd.read_sql(sql,conn)
  print(len(pdgrand))
  pdgrand.to_csv(r'./answer.csv',columns=['ID','Answer'],index=False,sep=',')
  conn.commit()
#產生答案
def genscore(conn):
  sql="SELECT ID, COALESCE( score1,0) as score1, COALESCE(score2,0) as score2,\
  COALESCE(score3,0) as score3 , COALESCE(score4,0) as score4 FROM Cpart ; "

  pdgrand=pd.read_sql(sql,conn)
  print(len(pdgrand))
  pdgrand.to_csv(r'./score.csv',columns=['ID','score1','score2','score3','score4'],index=False,sep=',')
  conn.commit()
def addpatch(conn):
##################################
#check 合併後少的檔案
##################################

  #蒐集目錄下wav檔名單
  wavlist = []
  directory = 'C/' 
  objects = os.listdir(directory)
  for filelist in objects:
    if filelist.find('.wav') != -1:
        wavlist.append(filelist)
       # print(filelist)
    wavlist_set=set(wavlist)

  #取得DB中的已辨識名單

  cursor = conn.cursor()
  sql="SELECT filename from  Cpart  ;" 
  cursor.execute(sql)
  dbvalues = list(cursor.fetchall())
  ldbvalues=[]
  for i in dbvalues:
    ldbvalues.append(i[0])
  sdbvalues=set(ldbvalues)

  #比較目錄名單及DB名單的差異
  diff=wavlist_set-sdbvalues

  #開啟新DB，將缺少的名單存入
  for filename in diff:
    try:
        ID=int(filename[4:8])
        sql="insert into Cpart (filename,ID) VALUES('%s',%d)"  %(filename,ID)
        print(sql)
        conn.execute(sql)
    except Exception as e:
        print("Failed insert db: "+ str(e)) 
        print(sql)
  conn.commit() 
   
#answettype =0 表示答案預設0,否則用random
def initanswer(conn,answertype):
    sql="select * from Cpart"
    pdgrand=pd.read_sql(sql,conn)
    answer=0
    for i in range(len(pdgrand)):
       filename=pdgrand.filename.values[i]
       if answertype!=0:
           answer=random.randint(1,4)   
       sql="update Cpart set Answer=%d where filename='%s'; " %(answer,filename)
      #print(sql)
       conn.execute(sql )
    conn.commit()
#把 cpart補到1500
addpatch(conn)
print('補完了')
initanswer(conn,1)
print('init finish')
genbtype(conn)

sql="SELECT apart.sentence,Cpart.sentence, c.filename as filename,c.qno as qno,count(*)\
    FROM Ccut c, acut a,Apart,Cpart where substr(a.filename,2,4)=substr(c.filename,5,4) and a.cut=c.cut \
and c.filename=cpart.filename and apart.filename=a.filename \
group by c.filename,qno order by c.filename,count(*) DESC "
#sqlpart a 改格式 
sql="SELECT apart.sentence,Cpart.sentence, c.filename as filename,c.qno as qno,count(*) as score\
    FROM Ccut c, acut a,Apart,Cpart where substr(a.filename,5,4)=substr(c.filename,5,4) and a.cut=c.cut \
and c.filename=cpart.filename and apart.filename=a.filename  \
group by c.filename,qno order by c.filename,count(*) DESC "
# bpart type 1正向答案

sql="SELECT C.ID,c.qno as qno,count(*) as score \
    FROM Ccut c, acut a,bpart b where a.id=c.id and a.cut=c.cut and a.id=b.id \
 and b.Qtype=1 group by c.ID,qno order by c.ID,count(*) DESC "

#print(sql)
pdgrand=pd.read_sql(sql,conn)
oldID=0
for i in range(len(pdgrand)):
   ID=pdgrand.ID.values[i]
   qno=pdgrand.qno.values[i]
   score=pdgrand.score.values[i] 
   sql="update Cpart set score%d=%d where ID =%d " %(qno,score,ID)
      #print(sql)
   conn.execute(sql )
   #sql="update list_A set Answer=%d where substr(filename,5,4) ='%s' " %(qno,filename[5:4])
   if ID!=oldID:
      sql="update Cpart set Answer=%d where ID =%d " %(qno,ID)
      #print(sql)
      conn.execute(sql )
    #  sql="update list_A set Answer=%d where substr(filename,5,4) ='%s' " %(qno,filename[5:4])
      #print(sql)
      conn.execute(sql )
      #conn.commit()
   oldID=ID

#for bpart type2反向問題

conn.commit()

sql="SELECT C.ID,c.qno as qno,count(*) as score \
    FROM Ccut c, acut a,bpart b where a.id=c.id and a.cut=c.cut and a.id=b.id \
 and b.Qtype=2 group by c.ID,qno order by c.ID,count(*) DESC "


#print(sql)
pdgrand=pd.read_sql(sql,conn)
oldID=0
for i in range(len(pdgrand)):
   ID=pdgrand.ID.values[i]
   qno=pdgrand.qno.values[i]
   score=-pdgrand.score.values[i] 
   sql="update Cpart set score%d=%d where ID =%d " %(qno,score,ID)
      #print(sql)
   conn.execute(sql )
   #sql="update list_A set Answer=%d where substr(filename,5,4) ='%s' " %(qno,filename[5:4])
 
   if ID!=oldID:
      sql="update Cpart set Answer=%d where ID =%d " %(qno,ID)
      #print(sql)
      conn.execute(sql )
    #  sql="update list_A set Answer=%d where substr(filename,5,4) ='%s' " %(qno,filename[5:4])
      #print(sql)
      conn.execute(sql )
      #conn.commit()
   oldID=ID

#for bpart type2反向問題
conn.commit()
print('answer complete')
gencsv(conn)
print('generate CSV file')
genscore(conn)
print('generate score file')
conn.close() 


In [ ]:
產生分數檔案
SELECT substr(filename,5,4) as ID, COALESCE( score1,0) as score1, COALESCE(score2,0) as score2, COALESCE(score3,0) as score3 , COALESCE(score4,0) as score4
FROM Cpart ;


In [ ]:
 找沒答案的sql
    SELECT cast(substr(a.filename,5,4) as int) ,a.answer,b.filename,b.answer
FROM list_A a  left join cpart b  on substr(a.filename,5,4)=substr(b.filename,5,4) order by b.answer
